<a href="https://colab.research.google.com/github/NicoPasche/WebScrappingUndefined_ba-/blob/main/botUndf.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Por Nicolas Paschetta, cualquier duda escriban a estos que respondo github@NicoPasche, ig@nicopaschet
Para que funcione desde el collab es CTRL + M + . (punto) -> CTRL + F9

In [ ]:
!pip install kora

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from kora.selenium import wd
import time
from bs4 import BeautifulSoup
import pandas as pd
import re
import numpy as np
import datetime
DATA_TALLES = ['XS','S','M','L','XL','XXL']
DATA_FUNDAS_COLOR = ['ROSA','NARANJA','VERDE','VIOLETA','ROJO']
DATA_FUNDAS_TAMANO = ['IPHONE 11','IPHONE 11 PRO','IPHONE 11 PRO MAX','IPHONE 12','IPHONE 12 PRO/IPHONE 12 PRO MAX','IPHONE 13 PRO','IPHONE 13 PRO MAX']
DATA_FUNDAS_TAMANO_2 = ['IPHONE 11','IPHONE 11 PRO','IPHONE 11 PRO MAX','IPHONE 12','IPHONE 12 PRO/IPHONE 12 PRO MAX','IPHONE 13','IPHONE 13 PRO','IPHONE 13 PRO MAX']
def extractor_url(itemListaURL):
  for itemURL in itemListaURL:
      if itemURL[0:4] == "http":
        return itemURL
def extractor_stock(stocks):
  for itemStock in stocks:
    if itemStock == "No":
      return 0
    elif itemStock == "None":
      return itemStock 
    elif len(itemStock)<=5:
      return int(itemStock)

def extractro_stock_individual(stockDeTodoSF):
  contador = 0
  row = []
  rows = []
  tabla = []
  for auxiliar in stockDeTodoSF[1:]:
    row.append(auxiliar_extracto_stock_individual(auxiliar))
    contador = contador + 1
  rows.append(row)
  if contador == 7:
    tabla = DATA_FUNDAS_TAMANO
  elif contador > 8:
    for i in DATA_FUNDAS_TAMANO_2:
      for j in DATA_FUNDAS_COLOR:
        k = i + " " +j 
        tabla.append(k)
  elif contador == 1:
    tabla.append("TALLE ÚNICO/ITEM ÚNICO")       
  else:
    tabla = DATA_TALLES[:contador]
  df = pd.DataFrame.from_records(rows,columns=tabla)
  return df
      
def auxiliar_extracto_stock_individual(auxiliar):
  valor = '' 
  for numero in auxiliar:
    if numero == ",":
      return int(valor) 
    else:
      valor = valor + numero 

def extraer_stocks(URL):
  wd.get(URL)
  soup = BeautifulSoup(wd.page_source)
  stocks = str(soup.find("meta",{"property":"tiendanube:stock"})).split('"')
  stock = extractor_stock(stocks)
  if stock == 'None':
    time.sleep(5)
    return extraer_stocks(URL)
  stockDeTodo = (soup.findAll("script",{"type":"text/javascript"}))
  stockDeTodo = ''.join(map(str,stockDeTodo))
  stockDeTodoSF = (str(stockDeTodo).split('"stock":'))
  stockIndividual = extractro_stock_individual(stockDeTodoSF)
  return [stock,stockIndividual]
def crear_tablas(URL):
  wd.get(URL)
  wd.set_window_size(1920,10080)
  soup = BeautifulSoup(wd.page_source)
  items = soup.find("div",{"class":"row-fluid container-wide container-xs container-product-grid"})
  rows_processed = []
  for item in items.findAll("div",{"class":"span3 item-container m-bottom-half"}):
    itemTitle = item.find('a',{"class":"js-item-name item-name"})
    itemTitle = itemTitle.text.strip('\n                                                    ')
    itemListaURL = str(item.find('a',{"title":itemTitle})).split('"')
    itemURL = extractor_url(itemListaURL)
    boolHayStock = bool(item.find('div',{"style":"display:none;"}))
    row = []
    row.append(itemTitle)
    if (boolHayStock):
      stock = extraer_stocks(itemURL)
      row.append(stock[0])
      row.append(stock[1].to_dict('records'))
    else:
      stock = 0
      row.append(stock)
      row.append(stock)
    rows_processed.append(row)  
  df = pd.DataFrame.from_records(rows_processed,columns=["Item","Total de Stock","Individuales"])
  return df
def checkeador(URL,tabla_uno,bandera = 1):
  while True:
    tabla_dos = crear_tablas(URL)
    auxiliar = tabla_dos
    #tabla_dos.to_csv('raw_data2.csv', index=False, header=True)
    if len(tabla_uno)>len(tabla_dos):
      cantidad = len(tabla_uno)-len(tabla_dos)
      print("Sacaron un/os item/s "+str(cantidad))
      bandera = 1
    elif len(tabla_uno)<len(tabla_dos):
      cantidad = len(tabla_dos)-len(tabla_uno)
      print("Agregaron un/os item/s "+str(cantidad))
      bandera = 1
    elif len(tabla_uno)==len(tabla_dos):
      tabla_dos['Cambios de Stock'] = np.where( (tabla_dos.sort_values(by=['Item'])['Total de Stock']) == (tabla_uno.sort_values(by=['Item'])['Total de Stock']), 'False', 'True')
      stock_anterior = tabla_uno['Total de Stock'] 
      stock_anterior = pd.Series(stock_anterior)
      tabla_dos = tabla_dos.assign(stock_anterior=stock_anterior.values)
      auxiliar2 = tabla_dos
      if len(auxiliar2.loc[tabla_dos['Cambios de Stock'] == 'True']) > 0 or bandera == 1:
        print(datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S\n'))
        tabla_dos.drop(tabla_dos.loc[tabla_dos['Total de Stock']==0].index, inplace=True)
        print(tabla_dos.sort_values(by=['Item'], ascending=False).to_string(index=False))
        bandera = 0
        
      del auxiliar2
      auxiliar.pop("Cambios de Stock")
      time.sleep(15)
    del tabla_uno
    tabla_uno = auxiliar
    del auxiliar
    del tabla_dos

In [ ]:
 def main():                                           #vamos a poder usar cualquier url de undf porque está hecho todo de la misma menera y el scrapping es igual para todos los productos 
  URL = "https://undefinedba.com/productos/?mpage=15"  #al ?mpage=n n siendo mayor al numero máximo así cargan todos los items y no hace falta hacerles scroll
  tabla_uno = crear_tablas(URL)                        #si llegan a poner un item con una nueva categoria, tipo 2,3 o 4 tipos se va a romper así que chiflenmé y lo modifico para esas cosas nuevas   
  checkeador(URL,tabla_uno)                            #le mandás un parametro más que sea equivalente a bandera = 0 si sabés que está cambiando todo el tiempo el stock, en este caso es para que a la primera siempre te tire el caso a sub cero 
  return 0                                                  


In [ ]:
if __name__ == '__main__':
  main()

2022-07-08 17:56:09

                    Item  Total de Stock                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                           